# Anotación automática de Rima con Rantanplan

- [Aplicar Rantanplan para la rimae](#rima-rantanplan)
  - [Extraer los patrones métricos a partir del resultado de Rantanplan](#extraer-escansion)
  - [Formatear los resultados](#formatear-resultados)
    - [Añadir el número del verso](#anadir-numero-verso)
    - [Añadir el texto de cada verso](#anadir-texto-verso)
- [Aplicar Rantanplan para la escansión de varios poemas en un directorio](#rima-rantanplan-directorio)

Este notebook muestra cómo utilizar [Rantanplan](https://github.com/linhd-postdata/rantanplan) para la anotación automática de la rima en español.

Rantanplan así como sus dependencias deben estar ya instalados. Se describe cómo hacerlo [aquí](https://github.com/HD-aula-Literatura/III-8-Verso/blob/main/etiquetado-escansion-rantanplan.ipynb)

<a name="rima-rantanplan"></a>

## Aplicar Rantanplan para la rima

Se da un ejemplo de código completo y a continuación su explicación

In [36]:
import rantanplan
from rantanplan.core import get_scansion

texto_poema = """Rosas sangrantes sobre el mar desflora
el sol que dice adioses en la tarde,
riman las aguas su canción sonora,
bajo nubes de fuego el poniente arde.

Vibran las cañas al chocar del viento,
formando extraña y triste sinfonía,
y la palmera altiva en vaivén lento
es una glauca nota de armonía.

Una barca se aleja lentamente,
una estela de luz, un vago canto,
sombras que pasan sobre el quieto mar;

Y las horas se van pausadamente,
mientras vierte la luz su último encanto
en un intenso, pálido llamear."""

resultado = get_scansion(texto_poema, rhyme_analysis=True)
for verso in resultado:
    try:
        identificador_rima = verso["rhyme"]
        rima = verso["ending"]
        tipo_rima = verso["rhyme_type"]
        # para sacar la palabra rima
        verso_sin_puntuacion = [token for token in verso["tokens"]
                                if "word" in token]
        palabra_rima = "".join([silaba["syllable"] for silaba
                                in verso_sin_puntuacion[-1]["word"]])
        # mostrar los resultados
        print(f"{identificador_rima}\t{rima}\t{palabra_rima}\t{tipo_rima}")
    except Exception as e:
        print(f"Error: {e}")

a	ora	desflora	consonant
b	arde	tarde	consonant
a	ora	sonora	consonant
b	arde	arde	consonant
c	ento	viento	consonant
d	ia	sinfonía	consonant
c	ento	lento	consonant
d	ia	armonía	consonant
e	ente	lentamente	consonant
f	anto	canto	consonant
g	ar	mar	consonant
e	ente	pausadamente	consonant
f	anto	encanto	consonant
g	ar	llamear	consonant


Ya se ha explicado [aquí](https://github.com/HD-aula-Literatura/III-8-Verso/blob/main/etiquetado-escansion-rantanplan.ipynb) en detalle la estructura de los resultados de Rantanplan, que consiste en una lista de diccionarios (una estructura de datos de Python que se organiza según pares de clave-valor. También se explicó en el mismo sitio cómo recorrer los resultados con un bucle `for`.

Nos concentramos aquí en cómo hacer que Rantanplan devuelva el análisis de rima y las nuevas claves que se añaden a sus diccionarios con los resultados de este análisis.

Para que Rantanplan efectúe el análisis, se llama a la función `get_scansion()` con un parámetro adicional, `rhyme_analysis=True`. Esto va a su vez añadir informaciones relativas a la rima en los diccionarios de salida. Para cada verso, encontraremos estas nuevas claves:

- `rhyme`: Posición del verso dentro del esquema rimático, expresada con una letra minúscula. P. ej. la primera rima se expresa como *a*, la segunda como *b* etc. Para un serventesio, como el del ejemplo, esto daría el esquema *abab*; en la aplicación el esquema rimático de los versos de arte mayor también se expresa con letras minúsculas.
- `ending`: Secuencia ortográfica que expresa la rima.
- `rhyme_type`: Consonante o asonante

Estas nuevas claves se han explotado en el bucle del código arriba. Sus valores se muestran en pantalla al incluirlos en la función `print` final.

En cuanto a la palabra rima, Rantanplan no la da directamente. Pero en el código de arriba se ha extraído esta palabra. En la clave `tokens` encontramos una clave `word` con diccionarios por sílaba, y en estos, la clave `syllable` da la secuencia ortográfica para cada verso. Teniendo en cuenta esto, se ha extraído la palabra rima en dos etapas:
- `verso_sin_puntuacion = [token for token in verso["tokens"] if "word" in token]`: Esto elimina los diccionarios de los tokens que no son una palabra, sino que corresponden a una puntuación. Los diccionarios de los tokens de tipo puntuación no tienen clave `word` (tienen una clave `symbol` en su lugar). Eliminamos los diccionarios que no tienen clave `word` y asignamos la lista de filtrada a `verso_sin_puntuacion`.
- `palabra_rima = "".join([silaba["syllable"] for silaba in verso_sin_puntuacion[-1]["word"]])`. El último diccionario de `verso_sin_puntuacion` corresponde a la última palabra del verso. Se accede al último elemento con `[-1]`. Dentro de este elemento, hay un diccionario por sílaba. Si juntamos con `"".join()` los valores de `syllable` de cada diccionario, obtenemos la palabra rima, que se ha asignado a `palabra_rima`.

Otro aspecto no visto anteriormente es la inclusión de la mayor parte del código dentro de un bloque `try ... except`. Esto va a imprimir un mensaje de error si el poema da error (porque la librería no consigue analizar la rima al no encontrar las palabras en sus recursos léxicos o por alguna otra razón). De no incluir el código dentro de `try ... except`, el programa se pararía en vez de imprimir un mensaje de error. Si analizamos un solo poema no hay mucha diferencia, pero si queremos analizar un corpus entero, veremos así los poemas bien procesados y los que dan error.



Se imprime a continuación el contenido de `resultado` para el último verso del bucle, para mostrar las nuevas claves relacionadas con la rima:

In [2]:
resultado

[{'tokens': [{'word': [{'syllable': 'Ro', 'is_stressed': True},
     {'syllable': 'sas', 'is_stressed': False, 'is_word_end': True}],
    'stress_position': -2},
   {'word': [{'syllable': 'san', 'is_stressed': False},
     {'syllable': 'gran', 'is_stressed': True},
     {'syllable': 'tes', 'is_stressed': False, 'is_word_end': True}],
    'stress_position': -2},
   {'word': [{'syllable': 'so', 'is_stressed': False},
     {'syllable': 'bre',
      'is_stressed': False,
      'has_synalepha': True,
      'is_word_end': True}],
    'stress_position': 0},
   {'word': [{'syllable': 'el', 'is_stressed': False, 'is_word_end': True}],
    'stress_position': 0},
   {'word': [{'syllable': 'mar', 'is_stressed': True, 'is_word_end': True}],
    'stress_position': -1},
   {'word': [{'syllable': 'des', 'is_stressed': False},
     {'syllable': 'flo', 'is_stressed': True},
     {'syllable': 'ra', 'is_stressed': False, 'is_word_end': True}],
    'stress_position': -2}],
  'phonological_groups': [{'sylla

<a name="formatear-resultados"></a>

### Formatear los resultados

<a name="anadir-numero-verso"></a>

#### Añadir el número del verso

Como se hizo en el caso de la escansión, podemos formatear los resultados de varias formas, como añadir el número de verso. Por qué esto funciona se vio en el notebook sobre escansión.

La novedad con respecto al código anterior es que se ha añadido un `0` inicial a los números de verso menores a 10, para que la alineación de las columnas sea más clara; esto se hace poniendo `{str.zfill(str(indice+1), 2)}`, que añade ceros a la izquierda de `indice+1` hasta ocupar `2` posiciones en vez de poner simplemente `{indice+1}` dentro del `print()` final.

In [29]:
for indice, verso in enumerate(resultado):
    try:
        identificador_rima = verso["rhyme"]
        rima = verso["ending"]
        tipo_rima = verso["rhyme_type"]
        # para sacar la palabra rima
        verso_sin_puntuacion = [token for token in verso["tokens"]
                                if "word" in token]
        palabra_rima = "".join([silaba["syllable"] for silaba
                                in verso_sin_puntuacion[-1]["word"]])
        # mostrar los resultados
        print(f"{str.zfill(str(indice+1), 2)}\t{identificador_rima}\t{rima}\t{palabra_rima}\t{tipo_rima}")
    except Exception as e:
        print(f"Error: {e}")

01	a	ora	desflora	consonant
02	b	arde	tarde	consonant
03	a	ora	sonora	consonant
04	b	arde	arde	consonant
05	c	ento	viento	consonant
06	d	ia	sinfonía	consonant
07	c	ento	lento	consonant
08	d	ia	armonía	consonant
09	e	ente	lentamente	consonant
10	f	anto	canto	consonant
11	g	ar	mar	consonant
12	e	ente	pausadamente	consonant
13	f	anto	encanto	consonant
14	g	ar	llamear	consonant


<a name="anadir-texto-verso"></a>

#### Añadir el texto de cada verso

La mecánica de esto se explicó en el notebook sobre escansión. Damos aquí el código aplicado al análisis de rima.

In [33]:
texto_sin_blancos = [verso for verso in texto_poema.split("\n") if verso != ""]
resultado_2 = get_scansion(texto_poema, rhyme_analysis=True)
for indice, verso in enumerate(resultado_2):
    try:
        identificador_rima = verso["rhyme"]
        rima = verso["ending"]
        tipo_rima = verso["rhyme_type"]
        # para sacar la palabra rima
        verso_sin_puntuacion = [token for token in verso["tokens"]
                                if "word" in token]
        palabra_rima = "".join([silaba["syllable"] for silaba
                                in verso_sin_puntuacion[-1]["word"]])
        # mostrar los resultados
        print(f"{str.zfill(str(indice+1), 2)}\t{identificador_rima}\t" + 
              f"{rima}\t{palabra_rima}\t{tipo_rima}\t{texto_sin_blancos[indice]}")
    except Exception as e:
        print(f"Error: {e}")


01	a	ora	desflora	consonant	Rosas sangrantes sobre el mar desflora
02	b	arde	tarde	consonant	el sol que dice adioses en la tarde,
03	a	ora	sonora	consonant	riman las aguas su canción sonora,
04	b	arde	arde	consonant	bajo nubes de fuego el poniente arde.
05	c	ento	viento	consonant	Vibran las cañas al chocar del viento,
06	d	ia	sinfonía	consonant	formando extraña y triste sinfonía,
07	c	ento	lento	consonant	y la palmera altiva en vaivén lento
08	d	ia	armonía	consonant	es una glauca nota de armonía.
09	e	ente	lentamente	consonant	Una barca se aleja lentamente,
10	f	anto	canto	consonant	una estela de luz, un vago canto,
11	g	ar	mar	consonant	sombras que pasan sobre el quieto mar;
12	e	ente	pausadamente	consonant	Y las horas se van pausadamente,
13	f	anto	encanto	consonant	mientras vierte la luz su último encanto
14	g	ar	llamear	consonant	en un intenso, pálido llamear.


<a name="rima-rantanplan-directorio"></a>


## Aplicar Rantanplan para analizar la rima de varios poemas (en un directorio)

Ahora reproducimos el procedimiento para obtener la rima de un poema, pero aplicándolo a un directorio que contiene un archivo por poema. Los resultados serán escritos a un único archivo de salida de tipo "dataframe", que se puede abrir con una hoja de cálculo (forma de proceder habitual para análisis de datos)

Esto requiere código para abrir ficheros y leer su contenido (que será ofrecido como entrada a Rantanplan para el análisis), además de código para escribir el archivo de salida con los resultados por verso.

Usaremos los poemas de [Evangelina Guerrero](https://github.com/pruizf/disco/tree/v4/txt/20th/per-author/disco004t), poeta filipina presente en el corpus DISCO.

El funcionamiento de este código se explicó en el notebook sobre escansión. Aquí nos limitamos a aplicarlo a la extracción de rima.


### Instalar librerías

Para guardar los datos en formato Excel, instalaremos `openpyxl` si no está ya instalado.

say what comments are
also write out the dataframe and on the book text give the screencap when you open it on a spreadsheet

In [64]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 2.0 MB/s eta 0:00:001.8 MB/s eta 0:00:01
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


### Aplicar Rantanplan

Se da primero el código completo y después explicaciones sobre su funcionamiento.

In [40]:
import pandas as pd
from pathlib import Path

import rantanplan
from rantanplan.core import get_scansion

dir_poemas = "/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t"
ruta_poemas = Path(dir_poemas)
ruta_salida = ruta_poemas / ".." / "rima_evangelina-guerrero.xlsx"

# preparar listas para los datos
poemas = []
numeros_poema = []
incipits = []
indices = []
identificadores_rima = []
rimas = []
tipos_rima = []
palabras_rima = []


# recorrer poemas y obtener información sobre rima
for numero_poema, ruta_poema in enumerate(ruta_poemas.iterdir()):
    # mostrar el nombre de archivo de cada poema en pantalla
    print(ruta_poema)
    # procesar cada poema
    with open(ruta_poema) as descriptor_poema:
        # filtrar las líneas vacías si las hay
        texto_sin_blancos_lista = [linea.strip() for linea in descriptor_poema
                                   if linea.strip() != ""]
        # añadir contenido de la lista con el texto de cada verso
        # a la lista general de datos
        poemas.extend(texto_sin_blancos_lista)
        # poner texto del poema como CADENA para dar a rantanplan
        texto_sin_blancos = "\n".join(texto_sin_blancos_lista)
        # rima (mismo proceso que para un solo poema)
        resultados = get_scansion(texto_sin_blancos, rhyme_analysis=True)
        for indice, verso in enumerate(resultados):
            try:
                identificador_rima = verso["rhyme"]
                rima = verso["ending"]
                tipo_rima = verso["rhyme_type"]
                # para sacar la palabra rima
                verso_sin_puntuacion = [token for token in verso["tokens"]
                                        if "word" in token]
                palabra_rima = "".join([silaba["syllable"] for silaba
                                        in verso_sin_puntuacion[-1]["word"]])
            except Exception as e:
                identificador_rima, rima, tipo_rima, palabra_rima = \
                    ("Error", "Error", f"Error: {e}", "Error")
            # añadir info de rima a lista general de datos
            identificadores_rima.append(identificador_rima)
            rimas.append(rima)
            tipos_rima.append(tipo_rima)
            palabras_rima.append(palabra_rima)
            # añadir número de cada verso, íncipit y
            # un número para el poema a lista general
            indices.append(indice+1)
            incipits.append(texto_sin_blancos_lista[0].strip())
            numeros_poema.append(numero_poema+1)
            

# crear dataframe con las listas generales de datos
datos = {"numPoema": numeros_poema,        
        "incipit": incipits,
        "numVerso": indices,
        "verso": poemas,
        "esquemaRima": identificadores_rima,
        "rima": rimas,
        "palabraRima": palabras_rima,
        "tipoRima": tipos_rima}

df = pd.DataFrame(datos)

# escribir la dataframe en el archivo de salida
df.to_excel(ruta_salida, index=False)

/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0090.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0094.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0093.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0091.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0096.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0095.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0089.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0092.txt


**Dataframe con los datos de salida**

In [41]:
df

,numPoema,incipit,numVerso,verso,esquemaRima,rima,palabraRima,tipoRima
0,1,Rosas sangrantes sobre el mar desflora,1,Rosas sangrantes sobre el mar desflora,a,ora,desflora,consonant
1,1,Rosas sangrantes sobre el mar desflora,2,"el sol que dice adioses en la tarde,",b,arde,tarde,consonant
2,1,Rosas sangrantes sobre el mar desflora,3,"riman las aguas su canción sonora,",a,ora,sonora,consonant
3,1,Rosas sangrantes sobre el mar desflora,4,bajo nubes de fuego el poniente arde.,b,arde,arde,consonant
4,1,Rosas sangrantes sobre el mar desflora,5,"Vibran las cañas al chocar del viento,",c,ento,viento,consonant
...,...,...,...,...,...,...,...,...
107,8,En el vaso sombrío de mis penas,10,Quedó un dulzor de mieles en mi boca,Error,Error,Error,Error: 'rhyme'
108,8,En el vaso sombrío de mis penas,11,y algo muriente se fundió en el viento.,Error,Error,Error,Error: 'rhyme'
109,8,En el vaso sombrío de mis penas,12,¡Ya dejé de llorar!... En mi horizonte,Error,Error,Error,Error: 'rhyme'
110,8,En el vaso sombrío de mis penas,13,vi dibujarse una esperanza loca,Error,Error,Error,Error: 'rhyme'


Al haber escrito la dataframe a un archivo, usando el formato Excel en este caso, podemos abrirla con un programa de hoja de cálculo. Los datos se muestran como en la imagen siguiente, abiertos con LibreOffice:

![Captura dataframe](./img/06_dataframe_corpus_rima_rantanplan.png)



